In [1]:
!pip install -q scikit-multiflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.6/450.6 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skmultiflow.drift_detection import ADWIN, DDM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import warnings
import scipy.io
import pickle
import sys
import gc
import os
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
code_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Codes/Shared Codes'
results_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Results'
feature_selection_results = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_results'
sys.path.insert(0,code_path)
from oselm import set_use_know
from SharedFunctions import main,generate_oselm_models

In [5]:
filenames = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','7recurrentDrift.csv', 'blip.csv', 'incrementalDrift.csv',
             '7gradualDrift.csv', '7suddenDrift.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))

In [6]:
data_name = ['kddcup99','ISCX2012','CSE-CIC2018','CICIDS2017','7recurrentDrift', 'blip', 'incrementalDrift','7gradualDrift', '7suddenDrift']

In [7]:
# # use_know = False
# # set_use_know(use_know)
# # path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
# # os.makedirs(path, exist_ok=True)
# # for f_name,d_name in zip(filenames,data_name):
# #   main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
# #        feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [8]:
DATA_NUMBER=0
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : kddcup99_csv.csv ======================


0it [00:00, ?it/s]

Evolving RandomForest
1 {'accuracy': 0.9961513790891597, 'precision': 0.997563946406821, 'recall': 0.9951397326852977, 'f1-score': 0.9963503649635036, 'auc': 0.996211170690475}


  0%|          | 0/7832 [00:00<?, ?it/s]

...

Evolving RandomForest
8 {'accuracy': 0.9518987341772152, 'precision': 0.9233449477351916, 'recall': 0.9875776397515528, 'f1-score': 0.9543817527010805, 'auc': 0.951208174714486}


  0%|          | 0/7892 [00:00<?, ?it/s]

Evolving RandomForest
9 {'accuracy': 0.8556048131728943, 'precision': 0.9915682967959528, 'recall': 0.7250308261405672, 'f1-score': 0.8376068376068377, 'auc': 0.8592602047369503}


  0%|          | 0/7790 [00:00<?, ?it/s]

Evolving RandomForest
10 {'accuracy': 0.8523748395378691, 'precision': 0.8008849557522124, 'recall': 0.9354005167958657, 'f1-score': 0.8629320619785459, 'auc': 0.8529043400305859}


In [9]:
DATA_NUMBER=1
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : ISCX2012.csv ======================


0it [00:00, ?it/s]

Evolving RandomForest
1 {'accuracy': 0.728402734617775, 'precision': 0.9939759036144579, 'recall': 0.43137254901960786, 'f1-score': 0.601640838650866, 'auc': 0.7145014403865813}


  0%|          | 0/8018 [00:00<?, ?it/s]

...

8 {'accuracy': 0.5037453183520599, 'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'auc': 0.5}


  0%|          | 0/8138 [00:00<?, ?it/s]

Evolving RandomForest
9 {'accuracy': 0.8101965601965602, 'precision': 0.9980544747081712, 'recall': 0.6248477466504263, 'f1-score': 0.7685393258426966, 'auc': 0.81180429463872}


  0%|          | 0/8122 [00:00<?, ?it/s]

Evolving RandomForest
10 {'accuracy': 0.7950769230769231, 'precision': 0.997979797979798, 'recall': 0.5980629539951574, 'f1-score': 0.7479182437547313, 'auc': 0.7984056947697938}


In [10]:
DATA_NUMBER=2
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : CSE-CIC2018.csv ======================


0it [00:00, ?it/s]

Evolving RandomForest
1 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'auc': 0.5}


  0%|          | 0/8000 [00:00<?, ?it/s]

...

Evolving RandomForest
8 {'accuracy': 0.9274092615769712, 'precision': 1.0, 'recall': 0.85297845373891, 'f1-score': 0.920656634746922, 'auc': 0.926489226869455}


  0%|          | 0/15962 [00:00<?, ?it/s]

Evolving RandomForest
9 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'auc': 1.0}


  0%|          | 0/15986 [00:00<?, ?it/s]

Evolving RandomForest
10 {'accuracy': 0.6879299562226392, 'precision': 1.0, 'recall': 0.36392606755895474, 'f1-score': 0.5336448598130842, 'auc': 0.6819630337794773}


In [11]:
DATA_NUMBER=3
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : CICIDS2017.csv ======================


0it [00:00, ?it/s]

Evolving RandomForest
1 {'accuracy': 0.9570720968629609, 'precision': 0.9647058823529412, 'recall': 0.9447004608294931, 'f1-score': 0.9545983701979046, 'auc': 0.9565441187182239}


  0%|          | 0/8974 [00:00<?, ?it/s]

...

Evolving RandomForest
8 {'accuracy': 0.8827160493827161, 'precision': 0.8180129990714949, 'recall': 0.9854586129753915, 'f1-score': 0.8939624556062912, 'auc': 0.8823689461273354}


  0%|          | 0/8874 [00:00<?, ?it/s]

Evolving RandomForest
9 {'accuracy': 0.8276056338028169, 'precision': 0.7414965986394558, 'recall': 0.9977116704805492, 'f1-score': 0.8507317073170731, 'auc': 0.8301543923989871}


  0%|          | 0/8898 [00:00<?, ?it/s]

Evolving RandomForest
10 {'accuracy': 0.7033707865168539, 'precision': 0.8257042253521126, 'recall': 0.522271714922049, 'f1-score': 0.6398362892223739, 'auc': 0.7050134084814327}


In [12]:
DATA_NUMBER=4
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : 7recurrentDrift.csv ======================


0it [00:00, ?it/s]

Evolving RandomForest
1 {'accuracy': 0.7492260061919505, 'precision': 0.7641154328732748, 'recall': 0.7372881355932204, 'f1-score': 0.7504621072088725, 'auc': 0.7495059182402097}


  0%|          | 0/8120 [00:00<?, ?it/s]

...

Evolving RandomForest
8 {'accuracy': 0.7351851851851852, 'precision': 0.7157079646017699, 'recall': 0.78998778998779, 'f1-score': 0.7510156703424259, 'auc': 0.7345694255806614}


  0%|          | 0/8176 [00:00<?, ?it/s]

Evolving RandomForest
9 {'accuracy': 0.7267726161369193, 'precision': 0.7202185792349727, 'recall': 0.7752941176470588, 'f1-score': 0.7467422096317281, 'auc': 0.7247971860499925}


  0%|          | 0/8082 [00:00<?, ?it/s]

Evolving RandomForest
10 {'accuracy': 0.7167594310451453, 'precision': 0.7076566125290024, 'recall': 0.7475490196078431, 'f1-score': 0.7270560190703218, 'auc': 0.7164711390174047}


In [13]:
DATA_NUMBER=5
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : blip.csv ======================


0it [00:00, ?it/s]

Evolving RandomForest
1 {'accuracy': 0.5032858707557503, 'precision': 0.5202020202020202, 'recall': 0.4387646432374867, 'f1-score': 0.47602541883304456, 'auc': 0.5051771355984502}


  0%|          | 0/8994 [00:00<?, ?it/s]

...

Evolving RandomForest
8 {'accuracy': 0.4932102118413905, 'precision': 0.49042709867452133, 'recall': 0.36195652173913045, 'f1-score': 0.41651031894934337, 'auc': 0.49313895576641653}


  0%|          | 0/9138 [00:00<?, ?it/s]

Evolving RandomForest
9 {'accuracy': 0.4824945295404814, 'precision': 0.4645061728395062, 'recall': 0.33444444444444443, 'f1-score': 0.3888888888888889, 'auc': 0.48026101532567045}


  0%|          | 0/9030 [00:00<?, ?it/s]

Evolving RandomForest
10 {'accuracy': 0.5155038759689923, 'precision': 0.5154109589041096, 'recall': 0.3370660694288914, 'f1-score': 0.4075829383886256, 'auc': 0.5135494640682244}


In [14]:
DATA_NUMBER=6
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : incrementalDrift.csv ======================


0it [00:00, ?it/s]

Evolving RandomForest
1 {'accuracy': 0.7262569832402235, 'precision': 0.743979721166033, 'recall': 0.7106537530266344, 'f1-score': 0.726934984520124, 'auc': 0.7266644561311516}


  0%|          | 0/8072 [00:00<?, ?it/s]

...

Evolving RandomForest
8 {'accuracy': 0.7237330037082819, 'precision': 0.7084257206208425, 'recall': 0.7764277035236938, 'f1-score': 0.7408695652173912, 'auc': 0.7228050467304005}


  0%|          | 0/8008 [00:00<?, ?it/s]

Evolving RandomForest
9 {'accuracy': 0.7434456928838952, 'precision': 0.7267303102625299, 'recall': 0.7699115044247787, 'f1-score': 0.7476979742173112, 'auc': 0.7437720284146089}


  0%|          | 0/8092 [00:00<?, ?it/s]

Evolving RandomForest
10 {'accuracy': 0.6201358863495985, 'precision': 0.6286173633440515, 'recall': 0.5045161290322581, 'f1-score': 0.5597709377236937, 'auc': 0.6154097232839015}


In [15]:
DATA_NUMBER=7
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : 7gradualDrift.csv ======================


0it [00:00, ?it/s]

Evolving RandomForest
1 {'accuracy': 0.6394891944990176, 'precision': 0.609375, 'recall': 0.7692307692307693, 'f1-score': 0.6800348735832608, 'auc': 0.6399969893120578}


  0%|          | 0/10100 [00:00<?, ?it/s]

...

Evolving RandomForest
8 {'accuracy': 0.6396841066140178, 'precision': 0.6028075970272502, 'recall': 0.745658835546476, 'f1-score': 0.6666666666666666, 'auc': 0.6431255018229037}


  0%|          | 0/10178 [00:00<?, ?it/s]

Evolving RandomForest
9 {'accuracy': 0.6429273084479371, 'precision': 0.628, 'recall': 0.7497612225405922, 'f1-score': 0.6835002176752285, 'auc': 0.6397946658709027}


  0%|          | 0/10160 [00:00<?, ?it/s]

Evolving RandomForest
10 {'accuracy': 0.6515748031496063, 'precision': 0.636, 'recall': 0.7585877862595419, 'f1-score': 0.6919060052219321, 'auc': 0.648094706137901}


In [16]:
DATA_NUMBER=8
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : 7suddenDrift.csv ======================


0it [00:00, ?it/s]

Evolving RandomForest
1 {'accuracy': 0.6246290801186943, 'precision': 0.5782575173477256, 'recall': 0.7796257796257796, 'f1-score': 0.6640106241699868, 'auc': 0.6317940218883615}


  0%|          | 0/10062 [00:00<?, ?it/s]

...

Evolving RandomForest
8 {'accuracy': 0.6361848574237955, 'precision': 0.6132596685082873, 'recall': 0.7565725413826679, 'f1-score': 0.6774193548387097, 'auc': 0.634989349142178}


  0%|          | 0/10216 [00:00<?, ?it/s]

Evolving RandomForest
9 {'accuracy': 0.702054794520548, 'precision': 0.6910490856592878, 'recall': 0.7137176938369781, 'f1-score': 0.702200488997555, 'auc': 0.7022345694618416}


  0%|          | 0/10234 [00:00<?, ?it/s]

Evolving RandomForest
10 {'accuracy': 0.6511968734733756, 'precision': 0.6153232242617718, 'recall': 0.7686939182452642, 'f1-score': 0.6835106382978724, 'auc': 0.6535040472452375}
